### Imports

In [2]:
import requests
from bs4 import BeautifulSoup
import psycopg2

### Connection Test

In [4]:
url = 'https://www.bundesliga.com/de/bundesliga/statistiken/clubs/passquote'
response = requests.get(url)

# Prüfe, ob die Anfrage erfolgreich war (Statuscode 200)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Hier kannst du die relevanten Daten aus der Webseite extrahieren, z.B.:
    results = soup.find_all('div', class_='result')
    statistics = soup.find_all('div', class_='statistics')
else:
    print('Fehler beim Abrufen der Webseite')

### PostgreSQL

In [ ]:
def connect_and_save_data(results, statistics):
    try:
        connection = psycopg2.connect(
            dbname='your_database_name',
            user='your_user',
            password='your_password',
            host='your_host',
            port='your_port'
                
            host="localhost",
            database="PSQL_ADSFS2023Gruppe15",
            user="ADSFS2023Gruppe15",
            password="ADS_FS_2023_G15!?"
        )

        cursor = connection.cursor()

        # Erstelle die Tabelle, falls sie noch nicht existiert
        create_table_query = '''
            CREATE TABLE IF NOT EXISTS bundesliga_results (
                id SERIAL PRIMARY KEY,
                result TEXT NOT NULL,
                statistics TEXT NOT NULL
            );
        '''
        cursor.execute(create_table_query)
        connection.commit()

        # Füge die extrahierten Daten in die Tabelle ein
        for result, statistic in zip(results, statistics):
            insert_data_query = '''
                INSERT INTO bundesliga_results (result, statistics) VALUES (%s, %s);
            '''
            cursor.execute(insert_data_query, (result.text, statistic.text))
        
        connection.commit()

    except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
    finally:
        # Schließe die Verbindung
        if connection:
            cursor.close()
            connection.close()

# Führe die Funktion mit den extrahierten Daten aus
connect_and_save_data(results, statistics)